Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt
%matplotlib inline

Setting the path and reading the train data set

In [2]:
os.getcwd()
os.chdir('C:\\Users\madhu\Verizon\FullDataSet')

In [3]:
vertrain=pd.read_csv('training_data.csv')
vertrain.head(5)

,id,ad,link,timestamp,checkout,order_placed,grp,funnel_level
0,session446676598812,ad412316860435,link1288490188810,2018-04-30T14:39:22.000-04:00,1,1,grp936302870529,lower
1,session60129542152,ad584115552306,link1288490188810,2018-04-30T08:39:23.000-04:00,1,1,grp936302870529,lower
2,session1511828488251,ad1056561954841,link1288490188810,2018-04-30T17:24:01.000-04:00,1,1,grp936302870529,lower
3,session1142461300754,ad1666447310876,link1288490188810,2018-04-30T10:42:57.000-04:00,1,1,grp936302870529,lower
4,session1142461300754,ad1666447310876,link1288490188810,2018-04-30T10:43:19.000-04:00,1,1,grp936302870529,lower


Cleaning the data

In [4]:
adnull=vertrain[vertrain.ad.isnull()]
adnotnull=vertrain[vertrain.ad.notnull()]
adnull_t1=adnotnull.groupby(['id','ad'])[['id','ad']].head()
adnull_t2=adnull_t1.sort_values(by=['id']).groupby(adnull_t1.id).first()[['ad']].reset_index()
adnull_fin=pd.merge(adnull,adnull_t2,on='id',how='left')
adnull_fin['ad']=adnull_fin['ad_y']
adnull_fin.drop(['ad_x','ad_y'],axis=1,inplace=True)
adnull_fin.head()

,id,link,timestamp,checkout,order_placed,grp,funnel_level,ad
0,session730144440347,link1288490188805,2018-04-30T12:42:52.000-04:00,1,0,grp627065225216,upper,ad1219770712094
1,session66,link1288490188816,2018-04-30T19:46:38.000-04:00,0,0,grp1340029796352,upper,NaN
2,session35,link1288490188816,2018-04-30T13:42:06.000-04:00,0,0,grp1340029796352,upper,ad566935683078
3,session57,link1288490188816,2018-04-30T18:19:18.000-04:00,1,0,grp1340029796352,upper,ad1340029796357
4,session67,link1288490188816,2018-04-30T19:54:48.000-04:00,1,1,grp1340029796352,upper,ad1365799600132


Merging the cleaned data to train dataset

In [5]:
vertrn=pd.concat([adnotnull,adnull_fin])
vertrn.head()

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,ad,checkout,funnel_level,grp,id,link,order_placed,timestamp
0,ad412316860435,1,lower,grp936302870529,session446676598812,link1288490188810,1,2018-04-30T14:39:22.000-04:00
1,ad584115552306,1,lower,grp936302870529,session60129542152,link1288490188810,1,2018-04-30T08:39:23.000-04:00
2,ad1056561954841,1,lower,grp936302870529,session1511828488251,link1288490188810,1,2018-04-30T17:24:01.000-04:00
3,ad1666447310876,1,lower,grp936302870529,session1142461300754,link1288490188810,1,2018-04-30T10:42:57.000-04:00
4,ad1666447310876,1,lower,grp936302870529,session1142461300754,link1288490188810,1,2018-04-30T10:43:19.000-04:00


creating features date time and hours from timestamp

In [6]:
vertrn['date']=[b[0:10].replace("-","") for b in vertrn['timestamp']]
vertrn['time']=[j[11:19].replace(":","") for j in vertrn['timestamp']]
vertrn['hours']=[l[20:29].replace("-","").replace(":","") for l in vertrn['timestamp']]
vertrn['date'].astype(int).head(5)
vertrn['time'].astype(int).head(2)
vertrn['hours'].astype(int).head(5)

0    400
1    400
2    400
3    400
4    400
Name: hours, dtype: int32

Key clicks differentiating between people ordered and not ordered

In [14]:
print(vertrn.loc[(vertrn['order_placed']==1) & (vertrn['funnel_level']=='lower')]['grp'].value_counts().head(5))
print(vertrn.loc[(vertrn['order_placed']==1) & (vertrn['funnel_level']=='lower')]['link'].value_counts().head(5))

grp111669149696     1779
grp498216206336     1273
grp317827579904     1203
grp979252543488     1098
grp1468878815232    1085
Name: grp, dtype: int64
link34359738385      1004
link523986010114      891
link1056561954816     874
link1271310319618     819
link1494648619010     810
Name: link, dtype: int64


In [15]:
print(vertrn.loc[(vertrn['order_placed']==0) & (vertrn['funnel_level']=='lower')]['grp'].value_counts().head(5))
print(vertrn.loc[(vertrn['order_placed']==0) & (vertrn['funnel_level']=='lower')]['link'].value_counts().head(5))

grp489626271744     936
grp704374636544     446
grp936302870529     398
grp206158430208     340
grp1468878815232    316
Name: grp, dtype: int64
link335007449089     697
link1314259992589    373
link1494648619010    261
link1056561954816    253
link678604832791     210
Name: link, dtype: int64


Groups and Links lead to dropout [Checkout =1 and order placed =0 ]

In [13]:
s=vertrn[(vertrn['checkout']==1)&(vertrn['order_placed']==0)].groupby('id')['time','hours'].max().reset_index()
print(pd.merge(vertrn,s,on=['id','time','hours'],how='right')['grp'].value_counts().head(5))
print(pd.merge(vertrn,s,on=['id','time','hours'],how='right')['link'].value_counts().head(5))

grp1511828488192    585
grp197568495616     226
grp489626271744     154
grp1511828488193    153
grp1056561954816    111
Name: grp, dtype: int64
link335007449089     97
link1348619730944    88
link68719476760      69
link687194767380     64
link1314259992589    61
Name: link, dtype: int64


Groups and Links lead to dropout [Checkout =0 and order placed =0 ]

In [16]:
u=vertrn[(vertrn['checkout']==0)&(vertrn['order_placed']==0)].groupby('id')['time','hours'].max().reset_index()
print(pd.merge(vertrn,u,on=['id','time','hours'],how='right')['grp'].value_counts().head(5))
print(pd.merge(vertrn,u,on=['id','time','hours'],how='right')['link'].value_counts().head(5))

grp1340029796352    5510
grp1056561954816    1336
grp936302870528      785
grp1589137899520     312
grp644245094400      272
Name: grp, dtype: int64
link214748364811     2553
link1468878815250    1000
link25769803776       770
link257698037775      549
link1194000908303     482
Name: link, dtype: int64


Links/Groups which are browsed at last in upper funnel(order placed=1)

In [21]:
t=vertrn[(vertrn['funnel_level']=='upper')&(vertrn['order_placed']==1)].groupby('id')['time','hours'].max().reset_index()
print(pd.merge(vertrn,t,on=['id','time','hours'],how='right')['grp'].value_counts().head(5))
print(pd.merge(vertrn,t,on=['id','time','hours'],how='right')['link'].value_counts().head(5))

grp1340029796352    652
grp1056561954816    182
grp936302870528      94
grp601295421440      45
grp1357209665537     44
Name: grp, dtype: int64
link1468878815250    173
link369367187470     169
link214748364811     135
link1194000908303     63
link257698037775      60
Name: link, dtype: int64


Links/Groups which are browsed at last in middle funnel(order placed=1)

In [22]:
w=vertrn[(vertrn['funnel_level']=='middle')&(vertrn['order_placed']==1)].groupby('id')['time','hours'].max().reset_index()
print(pd.merge(vertrn,w,on=['id','time','hours'],how='right')['grp'].value_counts().head(5))
print(pd.merge(vertrn,w,on=['id','time','hours'],how='right')['link'].value_counts().head(5))

grp1511828488192    920
grp197568495616     362
grp94489280512      183
grp1108101562368    165
grp1589137899520    116
Name: grp, dtype: int64
link25769803776      175
link1348619730944    113
link687194767380     102
link420906795010      77
link781684047909      53
Name: link, dtype: int64


Links/Groups which are browsed at first in lower funnel(order placed=1)

In [24]:
z=vertrn[(vertrn['funnel_level']=='lower')&(vertrn['order_placed']==1)].groupby('id')['time','hours'].min().reset_index()
print(pd.merge(vertrn,z,on=['id','time','hours'],how='right')['grp'].value_counts().head(5))
print(pd.merge(vertrn,z,on=['id','time','hours'],how='right')['link'].value_counts().head(5))

grp704374636544     533
grp283467841536     177
grp489626271744     103
grp970662608896      63
grp1279900254210     51
Name: grp, dtype: int64
link1314259992589    413
link343597383684     142
link1185410973706    120
link335007449089      82
link240518168585      63
Name: link, dtype: int64


Links/Groups which are browsed at last in lower funnel(order placed=1)

In [25]:
b=vertrn[(vertrn['funnel_level']=='lower')&(vertrn['order_placed']==1)].groupby('id')['time','hours'].max().reset_index()
print(pd.merge(vertrn,b,on=['id','time','hours'],how='right')['grp'].value_counts().head(5))
print(pd.merge(vertrn,b,on=['id','time','hours'],how='right')['link'].value_counts().head(5))

grp317827579904     954
grp438086664192      47
grp1709396983808     43
grp1090921693184     31
grp738734374912      22
Name: grp, dtype: int64
link1271310319618    658
link618475290626     216
link1649267441667     76
link609885356034      27
link481036337153      20
Name: link, dtype: int64
